# 要約 
このJupyter Notebookは、20の質問ゲームにおいて、人物、場所、物を特定するための大規模言語モデル（LLM）を使用したアプローチを提案しています。このプロジェクトでは、ターンごとのエージェントアプローチを採用し、質問を通じてターゲットキーワードを推測することを目指しています。

### 問題定義
- **問題**: 20の質問ゲームを通じて、キーワードのリストから特定の人物、場所、物を特定する。
- **アプローチ**: 大規模言語モデルを用い、GIS座標や製品レビューなどのデータを活用して、質問に対する適切な応答を導く。

### 考えられる解決策
1. **モデルのファインチューニング**: ランドマークの写真やGIS座標でファインチューニングすることで、より正確な質問応答を可能にする。
2. **テキストデータの活用**: 電話のテキスト会話データやウィキペディアの情報をReferし、特定の話し方やパターンを識別。
3. **検索信号の最適化**: セグメンテーションに基づき、質問範囲内での迅速なキーワード検索を可能にする非監視モデルを使用。

### コードの実装
- **データ読み込み**: JSON形式のキーワードリストを読み込み、データフレームに格納する。
- **エージェントの設計**: 質問者（QuestionerAgent）と回答者（AnswererAgent）を定義し、それぞれの役割に基づいてレスポンスを生成するメソッドを実装。
- **モデルの初期化**: GemmaのカジュアルLMモデルを使用し、エージェントが機能するように構成する。
- **環境テスト**: Kaggleの環境を構築し、4つのエージェントが対戦するテストを実行する。

### 使用したライブラリ
- **Pandas**: データフレームの操作。
- **Torch**: モデルの実装。
- **Kaggle Environments**: ゲーム環境を構築し、エージェントの対戦を可能にする。

このNotebookは、20の質問ゲームをプレイし、エージェントを通じてでキーワードを特定するための新しいアプローチを提示しており、戦略的な質問応答を通じた推論能力の評価を可能にします。

---


# 用語概説 
以下は、提供されたJupyter Notebookに含まれる専門用語や概念の簡単な解説です。特に機械学習や深層学習の初心者がつまずきやすい、あまり知られていない用語や特定のドメイン知識に焦点を当てています。

1. **GIS座標 (Geographic Information System Coordinates)**:
   地理情報システム (GIS) とは、地理データを収集、保存、解析、表示するためのシステムです。GIS座標は、特定の地点を地球上の位置で表現するために使用される緯度と経度の組み合わせです。物理的な場所を特定するのに役立ちます。

2. **ファインチューニング (Fine-tuning)**:
   既に学習済みのモデルを特定のタスクに適応させるプロセスです。新しいデータでモデルを再学習させて精度を向上させる方法で、通常、完全な再トレーニングよりも早く、少ないデータで行われます。

3. **検索信号 (Search Signal)**:
   検索信号とは、特定の情報を探す際に役立つデータや特徴のことです。ここでは、質問者がキーワードを見つけるために必要な情報を指します。

4. **逆引き (Reverse Lookup)**:
   データベースやデータ構造において、特定のデータに基づいて関連する情報を見つけるプロセスです。ここでは、現在の使用モデルの重みや埋め込みを利用して、特定の質問に対する適切な回答を生成する方法を指します。

5. **トークン (Token)**:
   自然言語処理において、トークンは文章を構成する最小の単位です。単語や句読点などが該当し、モデルがテキストを理解するために用います。

6. **モデルの重み (Model Weights)**:
   ニューラルネットワークの各接続に付与される数値で、学習によって最適化されます。モデルの重みが異なると、同じ入力に対して異なる出力が得られます。

7. **非監視モデル (Unsupervised Model)**:
   ラベル付けされたデータを使用せずにデータのパターンや構造を学習するモデルです。クラスタリングや次元削減のタスクに一般的に使用されます。

8. **エージェント (Agent)**:
   特定のタスクを実行するために設計されたソフトウェアプログラムまたはモデルです。このノートブックでは、「質問者」エージェントと「回答者」エージェントがそれぞれ質問と回答を担当します。

9. **温度 (Temperature)**:
   確率分布のスプレッドを調整するパラメータで、この値が高いほど多様性のある出力が得られ、低いほど決定論的な出力になる傾向がある。生成モデルでは、創造性と一貫性のバランスを取るために使用されます。

10. **少数ショット学習 (Few-shot Learning)**:
    限られたデータポイントでモデルをトレーニングし、新しいタスクを学習させる能力です。あまりデータがない状況で有用であり、一般化能力を示します。

これらの用語が初心者にとって参考になり、Jupyter Notebookへの理解を深める助けとなることを願っています。

---


## 問題定義:
20の質問ゲームをプレイしながら、キーワードのリストから人物、場所、物を特定するために、大規模言語モデルまたは新しいプログラミングアプローチを使用します。ターンごとのエージェントアプローチを採用します。  

考えられる解決策:

* 大規模言語モデルは、国、都市、州、そしてランドマークの近似を質問に対してより適切に調整するために、写真からのランドマークや都市のGIS座標の中心点でファインチューニングすることができます。

* 電話のテキスト会話のデータセットにGIS座標を使用することも可能であり、変換された場所を用いて言語のローカリゼーション、特定の話し方のパターンや口語を特定することができるかもしれません。

* 物に関しては、製品説明や製品レビューを使用することができるかもしれません。

* 人に関しては、ウィキペディアやLinkedIn、デーティングサイトなどのソーシャルネットワーキングを参照することができます。

* ファインチューニングなしの他のアプローチとしては、20の質問範囲内でキーワードにすぐに到達できるように、セグメンテーションに対して優れた検索信号を生み出すトークンを特定することができ、現在の使用モデルの重みや埋め込みに基づいて逆引きする非監視モデルに利用できます。

例:
* https://www.cia.gov/the-world-factbook/
* https://public-nps.opendata.arcgis.com/datasets/nps::national-register-of-historic-places-points/explore


## キーワードリスト


In [ ]:
import pandas as pd
import json

# モジュールを動的に読み込みます
from importlib.machinery import SourceFileLoader
# keywords.pyからKEYWORDS_JSONをロードします
keywords = SourceFileLoader("keywords",'/kaggle/input/llm-20-questions/llm_20_questions/keywords.py').load_module()
# JSON形式のキーワードデータを読み込みます
df = json.loads(keywords.KEYWORDS_JSON)
words = []

# 各カテゴリーについてループ処理を行います
for c in df:
    # カテゴリー名と単語数を出力します
    print(c['category'], len(c['words']))
    # カテゴリー内の各単語についてさらにループ処理を行います
    for w in c['words']:
        # カテゴリー、キーワード、代替語、空のフィールド、緯度、経度をwordsリストに追加します
        words.append([c['category'], w["keyword"], w["alts"], "", "", 0.0, 0.0])

# データフレームを作成し、カラム名を設定します
df = pd.DataFrame(words, columns=['Category','Word','Alternatives', "Cat1", "Cat2", "Lat", "Lon"])
# データフレームの最終5行を表示します
df.tail()

## テンプレートフォーマッティング
* https://www.promptingguide.ai/models/gemma

In [ ]:
template_special_tokens = ['<bos>','<start_of_turn>user','<start_of_turn>model','<end_of_turn>','<eos>']

# エージェント1 - 推測者 (モデル？)
# エージェント2 - 回答者 (ユーザー？)
# エージェント3 - 推測者 (モデル？)
# エージェント4 - 回答者 (ユーザー？)

def agentx(obs, cfg):
    # 観察結果のターンタイプに基づいてレスポンスを生成します
    if obs.turnType == "ask": 
        response = ""  # 質問のターンの場合のレスポンス
    elif obs.turnType == "guess": 
        response = ""  # 推測のターンの場合のレスポンス
    elif obs.turnType == "answer": 
        response = ""  # 回答のターンの場合のレスポンス
    else: 
        response = "Am I Halucinating?  **YES**"  # 他の不明なターンタイプに対するレスポンス
    return response  # 生成したレスポンスを返します

## 20の質問提出モデル

In [ ]:
%%bash
# 作業ディレクトリを/kaggle/workingに移動します
cd /kaggle/working
# immutabledictとsentencepieceの最新バージョンをインストールします
pip install -q -U -t /kaggle/working/submission/lib immutabledict sentencepiece
# gemma_pytorchリポジトリをクローンします
git clone https://github.com/google/gemma_pytorch.git > /dev/null
# gemma用のディレクトリを作成します
mkdir /kaggle/working/submission/lib/gemma/
# クローンしたgemmaのファイルを新しいディレクトリに移動します
mv /kaggle/working/gemma_pytorch/gemma/* /kaggle/working/submission/lib/gemma/

In [ ]:
%%writefile submission/main.py

import torch, itertools, contextlib
import os, sys, re, gc
from typing import Iterable
from pathlib import Path
gc.enable()

# Kaggleエージェントパスを設定します
KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
if os.path.exists(KAGGLE_AGENT_PATH):
    sys.path.insert(0, os.path.join(KAGGLE_AGENT_PATH, 'lib'))
    WEIGHTS_PATH = os.path.join(KAGGLE_AGENT_PATH, "gemma/pytorch/2b-it/2")
else:
    sys.path.insert(0, "/kaggle/working/submission/lib")
    WEIGHTS_PATH = "/kaggle/input/gemma/pytorch/2b-it/2"

from gemma.config import get_config_for_2b
from gemma.model import GemmaForCausalLM

# これは短縮され、ゲームメモリストレージで補完できます open('gamemaster.dat'), open('player.dat')
class AgentFormatter:
    def __init__(self, sp: str = None, fse: Iterable = None):
        self._system_prompt = sp
        self._few_shot_examples = fse
        self._turn_user = f"<start_of_turn>user\n{{}}<end_of_turn>\n"
        self._turn_model = f"<start_of_turn>model\n{{}}<end_of_turn>\n"
        self.reset()
        
    def __repr__(self):
        return self._state
    
    def user(self, prompt):
        self._state += self._turn_user.format(prompt)
        return self
    
    def model(self, prompt):
        self._state += self._turn_model.format(prompt)
        return self
    
    def reset(self):
        self._state = ""
        if self._system_prompt is not None:
            self.user(self._system_prompt)
        if self._few_shot_examples is not None:
            self.apply_turns(self._few_shot_examples, start_agent='user')
        return self
    
    def apply_turns(self, turns: Iterable, start_agent: str):
        formatters = [self.model, self.user] if start_agent == 'model' else [self.user, self.model]
        formatters = itertools.cycle(formatters)
        for fmt, turn in zip(formatters, turns):
            fmt(turn)
        return self

@contextlib.contextmanager
def _set_default_tensor_type(dtype: torch.dtype):
    torch.set_default_dtype(dtype)
    yield
    torch.set_default_dtype(torch.float)

class Agent:
    def __init__(self, sp=None, fse=None):
        self._device = torch.device('cuda:0')
        self.formatter = AgentFormatter(sp=sp, fse=fse)
        print("モデルの初期化中")
        model_config = get_config_for_2b()
        model_config.tokenizer = WEIGHTS_PATH + '/tokenizer.model'
        with _set_default_tensor_type(model_config.get_dtype()):
            model = GemmaForCausalLM(model_config)
            model.load_weights(WEIGHTS_PATH + '/gemma-2b-it.ckpt')
            self.model = model.to(self._device).eval()
            
    def __call__(self, obs, *args):
        self._start_session(obs)
        prompt = str(self.formatter)
        response = self._call_llm(prompt)
        response = self._parse_response(response, obs)
        print(obs.turnType, response)
        return response
    
    def _call_llm(self, prompt, max_nt=40, **sampler_kwargs):
        if sampler_kwargs is None:
            sampler_kwargs = {'temperature': 0.9, 'top_p': 0.9, 'top_k': 50,}
        response = self.model.generate(
            prompt, device=self._device, output_len=max_nt, **sampler_kwargs,)
        return response
    
    def _parse_keyword(self, response: str):
        match = re.search(r"(?<=\*\*)([^*]+)(?=\*\*)", response) #キーワードリストで最も近い一致をチェックし、正規表現を更新する
        if match is None: keyword = ''
        else: keyword = match.group().lower()
        return keyword
    
    def _parse_response(self, response: str, obs: dict):
        if obs.turnType == 'ask':
            match = re.search(".+?\?", response.replace('*', ''))
            if match is None: question = "Is it a place?" #人物、場所、物のランダムな選択をする
            else: question = match.group()
            return question
        elif obs.turnType == 'guess':
            guess = self._parse_keyword(response)
            if guess is None or len(guess) <= 1: 
                return "no guess" #ランダムキーワード？
            else: 
                return guess
        elif obs.turnType == 'answer':
            answer = self._parse_keyword(response)
            return 'yes' if 'yes' in answer else 'no'
        else: raise ValueError("不明なターンタイプ:", obs.turnType)

def interleave_unequal(x, y):
    return [item for pair in itertools.zip_longest(x, y) for item in pair if item is not None]

class QuestionerAgent(Agent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def _start_session(self, obs):
        self.formatter.reset()
        self.formatter.user("あなたは20の質問ゲームで質問者の役割を果たしています。")
        turns = interleave_unequal(obs.questions, obs.answers)
        self.formatter.apply_turns(turns, start_agent='model')
        if obs.turnType == 'ask':
            self.formatter.user("あなたのターンは、人物、場所、物を探すために「はい」または「いいえ」で尋ねることです。")
        elif obs.turnType == 'guess':
            self.formatter.user("あなたのターンは、人物、場所、物を推測することです。推測を二重アスタリスクで囲い、**yes**または**no**としてください。")

    def _parse_response(self, response: str, obs: dict):
        if obs.turnType == 'ask':
            match = re.search(".+?\?", response.replace('*', ''))
            if match is None: question = "Is it a place?" #人物、場所、物のランダムな選択をする
            else: question = match.group()
            return question
        elif obs.turnType == 'guess':
            guess = self._parse_keyword(response)
            if guess is None or len(guess) <= 1: 
                return "no guess" 
            else: 
                return guess
        elif obs.turnType == 'answer':
            answer = self._parse_keyword(response)
            return 'yes' if 'yes' in answer else 'no'
        else: raise ValueError("不明なターンタイプ:", obs.turnType)

class AnswererAgent(Agent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def _start_session(self, obs):
        self.formatter.reset()
        self.formatter.user(f"あなたは20の質問ゲームで回答者の役割を果たしています。キーワードは{obs.keyword}で、カテゴリーは{obs.category}です。")
        turns = interleave_unequal(obs.questions, obs.answers)
        self.formatter.apply_turns(turns, start_agent='user')
        self.formatter.user(f"質問はキーワード{obs.keyword}についてで、カテゴリーは{obs.category}です。「はい」または「いいえ」と答え、二重アスタリスクで囲んでください（例：**yes**または**no**）.")

    def _parse_response(self, response: str, obs: dict):
        answer = self._parse_keyword(response)
        return 'yes' if 'yes' in answer else 'no'

sp = "あなたは20の質問ゲームをプレイしています。このゲームでは、回答者がキーワードを考え、その後質問者の「はい」または「いいえ」の質問に答えます。キーワードは特定の人物、場所、または物です。"
fse = [
    "あなたは20の質問ゲームで質問者の役割を果たしています。最初の質問をして、人物、場所、物のキーワードを推測してください。",
    "それは人物ですか？", "**no**",
    "それは場所ですか？", "**yes**",
    "それは国ですか？", "**yes** さて、キーワードを推測してください。",
    "**フランス**", "正解!",
]
agent = None

def get_agent(name: str):
    global agent
    if agent is None and name == 'questioner':
        agent = QuestionerAgent(sp=sp, fse=fse,)
    elif agent is None and name == 'answerer':
        agent = AnswererAgent(sp=sp, fse=fse,)
    assert agent is not None, "エージェントが初期化されていません。"
    return agent

def agent_fn(obs, cfg):
    if obs.turnType == "ask": response = get_agent('questioner')(obs)
    elif obs.turnType == "guess": response = get_agent('questioner')(obs)
    elif obs.turnType == "answer": response = get_agent('answerer')(obs)
    
    if response is None or len(response) <= 1: return "no" #デフォルトを変更
    else: return response

# テスト

In [ ]:
!pip install -q pygame
!pip install -q 'kaggle_environments>=1.14.8'  # 新しい環境向けにKaggle環境をインストールします

In [ ]:
# コードを実行します
%run submission/main.py

In [ ]:
from kaggle_environments import make

# Kaggle環境を作成します
env = make("llm_20_questions", debug=True)
agent = "/kaggle/working/submission/main.py"

# 環境をリセットします
env.reset()
# エージェントを使って環境を実行します
logs = env.run([agent, agent, agent, agent])
# while not env.done: # テスト用に追加のステップをここに入れることができます
# 環境の結果を表示します
env.render(mode="ipython", width=800, height=800)

# パッケージ

In [ ]:
!apt install pigz pv > /dev/null  # pigzとpvをインストールします

In [ ]:
!tar --use-compress-program='pigz --fast --recursive | pv' -cf submission.tar.gz -C /kaggle/working/submission . -C /kaggle/input/ gemma/pytorch/2b-it/2  # submissionディレクトリを圧縮してtar.gzファイルを作成します

---

# コメント 

> ## DataDiva007
> 
> ここでデータサイエンスプロジェクトに取り組むのがすごく中毒的です。とても楽しい時間を過ごしています！
> 
> 
> 

---

> ## SuM
> 
> 素晴らしい仕事ですね [@jazivxt](https://www.kaggle.com/jazivxt)🎉
> 
> 共有してくれてありがとう、スタートするための重要なインスピレーションを与えてくれます😃
> 
> 

---

> ## Neel Patel
> 
> すごい仕事ですね、提出前にモデルをテストするのに非常に役立ちます。 :)

> 
---

> ## Hassan shahidi
> 
> 優れたノートブックです [@jazivxt](https://www.kaggle.com/jazivxt)
> 
> 

---

> ## Prajwal Kanade
> 
> 素晴らしく、非常に有益です、
> 
> 優れた仕事です [@jazivxt](https://www.kaggle.com/jazivxt) 
> 
> 

---

> ## Utkarsh Jain
> 
> 興味深いノートブック!! とても洞察に満ちています!!
> 
> 

---

> ## huoyeqianxun
> 
> CUDAのメモリ不足のようです…
> 
> ノートブックで4つのエージェントを実行できませんか？？？
> 
> 

---